In [48]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from textblob import Word
from sklearn import preprocessing, linear_model, metrics
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from xgboost import XGBClassifier
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image

In [3]:
train = pd.read_csv("C:/DATASETS/sentiment analysis/train.csv")
test = pd.read_csv("C:/DATASETS/sentiment analysis/test.csv")

In [11]:
train.head()

,id,label,tweet
31957,31958,0,ate @user isz that youuu?ðððððð...
31958,31959,0,to see nina turner on the airwaves trying to...
31959,31960,0,listening to sad songs on a monday morning otw...
31960,31961,1,"@user #sikh #temple vandalised in in #calgary,..."
31961,31962,0,thank you @user for you follow


In [5]:
test.head()

,id,tweet
0,31963,#studiolife #aislife #requires #passion #dedic...
1,31964,@user #white #supremacists want everyone to s...
2,31965,safe ways to heal your #acne!! #altwaystohe...
3,31966,is the hp and the cursed child book up for res...
4,31967,"3rd #bihday to my amazing, hilarious #nephew..."


In [6]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31962 entries, 0 to 31961
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      31962 non-null  int64 
 1   label   31962 non-null  int64 
 2   tweet   31962 non-null  object
dtypes: int64(2), object(1)
memory usage: 749.2+ KB


In [7]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17197 entries, 0 to 17196
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17197 non-null  int64 
 1   tweet   17197 non-null  object
dtypes: int64(1), object(1)
memory usage: 268.8+ KB


In [10]:
train.groupby("label").count()

,id,tweet
label,,
0,29720,29720
1,2242,2242


In [12]:
train["tweet"] = train["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))
test["tweet"] = test["tweet"].apply(lambda x: " ".join(x.lower() for x in x.split()))

# REMOVING PUNCTUATION MARKS AND NUMBERS

In [13]:
train["tweet"] = train["tweet"].str.replace('[^\w\s]','')
test["tweet"] = test["tweet"].str.replace('[^\w\s]','')

C:\Users\chenn\AppData\Local\Temp\ipykernel_3236\1362411275.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train["tweet"] = train["tweet"].str.replace('[^\w\s]','')
C:\Users\chenn\AppData\Local\Temp\ipykernel_3236\1362411275.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test["tweet"] = test["tweet"].str.replace('[^\w\s]','')


In [20]:
test.head()

,id,tweet
0,31963,studiolife aislife requires passion dedication...
1,31964,user white supremacists want everyone see new ...
2,31965,safe ways heal acne altwaystoheal healthy healing
3,31966,hp cursed child book reservations already yes ...
4,31967,rd bihday amazing hilarious nephew eli ahmir u...


In [15]:
train["tweet"] = train["tweet"].str.replace('\d',' ')
test["tweet"] = test["tweet"].str.replace('\d',' ')

C:\Users\chenn\AppData\Local\Temp\ipykernel_3236\1585875343.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  train["tweet"] = train["tweet"].str.replace('\d',' ')
C:\Users\chenn\AppData\Local\Temp\ipykernel_3236\1585875343.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  test["tweet"] = test["tweet"].str.replace('\d',' ')


# Removing stopwords and lemmatization

In [19]:
sw = stopwords.words("english")
train['tweet'] = train['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))
test['tweet'] = test['tweet'].apply(lambda x: " ".join(x for x in x.split() if x not in sw))


In [23]:
train['tweet'] = train['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
test['tweet'] = test['tweet'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()])) 


In [24]:
train = train.drop("id", axis = 1)
tes = test.drop("id", axis = 1)

# SPLITTING THE DATASET

In [29]:
x = train["tweet"]
y = train["label"]

train_x, test_x, train_y, test_y = train_test_split(x, y, test_size = 0.20, shuffle = True, random_state = 11)


In [32]:
vectorizer = CountVectorizer()
vectorizer.fit(train_x)

x_train_count = vectorizer.transform(train_x)
x_test_count = vectorizer.transform(test_x)

x_train_count.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [33]:
tf_idf_word_vectorizer = TfidfVectorizer()
tf_idf_word_vectorizer.fit(train_x)

x_train_tf_idf_word = tf_idf_word_vectorizer.transform(train_x)
x_test_tf_idf_word = tf_idf_word_vectorizer.transform(test_x)

x_train_tf_idf_word.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

# TRAINING  LOGISTIC REGRESSION UNDER countvectors and tf_idf

In [35]:
log = linear_model.LogisticRegression()
log_model = log.fit(x_train_count, train_y)
accuracy = model_selection.cross_val_score(log_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

C:\Users\chenn\Documents\python_jupyter\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [36]:
print(f"Accuracy : {accuracy}")

Accuracy : 0.947755681818182


In [37]:
log = linear_model.LogisticRegression()
log_model = log.fit(x_train_tf_idf_word, train_y)
accuracy = model_selection.cross_val_score(log_model,
                                           x_test_tf_idf_word,
                                           test_y,
                                           cv = 20).mean()

In [38]:
print(f"Accuracy : {accuracy}")

Accuracy : 0.9308635384012538


# TRAINING  XGBOOST UNDER countvectors and tf_idf

In [44]:
xgb = XGBClassifier()
xgb_model = xgb.fit(x_train_count,train_y)
accuracy = model_selection.cross_val_score(xgb_model,
                                           x_test_count,
                                           test_y,
                                           cv = 20).mean()

In [45]:
print(f"Accuracy : {accuracy}")

Accuracy : 0.9421287225705329


In [46]:
xgb = XGBClassifier()
xgb_model = xgb.fit(x_train_tf_idf_word,train_y)
accuracy = model_selection.cross_val_score(xgb_model, 
                                           x_test_tf_idf_word, 
                                           test_y, 
                                           cv = 20).mean()


In [47]:
print(f"Accuracy : {accuracy}")

Accuracy : 0.9404104623824454


In [49]:
tw_mask = np.array(Image.open('../input/masksforwordclouds/twitter_mask3.jpg'))

text = " ".join(i for i in train_set.tweet)

wc = WordCloud(background_color = "white",
               width = 600, mask = tw_mask,
               height = 600,
               contour_width = 0,
               contour_color = "red",
               max_words = 1000,
               scale = 1,
               collocations = False,
               repeat = True,
               min_font_size = 1)

wc.generate(text)

plt.figure(figsize = [15, 15])
plt.imshow(wc)
plt.axis("off")
plt.show;

FileNotFoundError: [Errno 2] No such file or directory: '../input/masksforwordclouds/twitter_mask3.jpg'